In [1]:
using CSV, DataFrames, JuMP, Gurobi, StatsPlots, Random, Statistics

## Load in data

In [2]:
# Define sizes
#num_suppliers = 200 # i
#num_consumer_regions = 7 # j = c(i) 
#num_time = 10; # t 
#num_producer_regions = 5 # p(i)

# Parameters
# alpha
# delta

In [3]:
availability = CSV.read("availability.csv", DataFrame)
col_order = [1, 9, 8, 7, 6, 5, 4, 3, 2]
availability = availability[:, col_order]
availability = sort(availability, :Region)

Row,Region,yr_2016_shoes,yr_2017_shoes,yr_2018_shoes,yr_2019_shoes,yr_2020_shoes,yr_2021_shoes,yr_2022_shoes,yr_2023_shoes
,String15,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,AMERICAS,4.18205e6,4.41752e6,4.80516e6,4.62324e6,5.5588e6,5.78138e6,6.57332e6,3.94215e6
2,EMEA,6.20928e5,6.5589e5,7.13444e5,6.86434e5,8.25341e5,8.58389e5,9.75971e5,5.85558e5
3,N ASIA,2.46451e7,2.60328e7,2.83172e7,2.72451e7,3.27584e7,3.40701e7,3.87371e7,2.32304e7
4,S ASIA,6.5423e7,6.91067e7,7.51708e7,7.2325e7,8.69607e7,9.04427e7,1.02832e8,6.16678e7
5,SE ASIA,8.60817e7,9.09287e7,9.89076e7,9.51631e7,1.1442e8,1.19002e8,1.35303e8,8.11406e7


In [4]:
demand = CSV.read("demand.csv", DataFrame)
replace!(demand.Region, "Europe, Middle East, and Africa" => "EMEA")
demand

Row,Region,2016,2017,2018,2019,2020,2021,2022,2023
,String31,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,Asia Pacific,1.84351e7,2.06687e7,2.24933e7,2.27891e7,2.17006e7,2.30219e7,2.58658e7,2.85839e7
2,EMEA,4.32876e7,4.45665e7,5.04292e7,5.40172e7,5.05751e7,5.98283e7,6.34163e7,7.09013e7
3,Greater China,2.2309e7,2.50644e7,3.00086e7,3.65837e7,3.97854e7,4.93391e7,4.64893e7,4.66524e7
4,Latin America,6.71511e6,7.52871e6,8.19335e6,8.30106e6,7.90458e6,8.38586e6,9.42178e6,1.04119e7
5,North America,7.98197e7,8.31245e7,8.00172e7,8.62232e7,8.00773e7,9.99485e7,1.04961e8,1.27871e8


In [5]:
revenue = CSV.read("rev.csv", DataFrame)
replace!(revenue.Region, "Europe, Middle East, and Africa" => "EMEA")
rev_year = []
for i in 2:size(revenue)[2]
    col_sum = sum(revenue[:, i]) * 1e6
    append!(rev_year, col_sum)
end
rev_year = Array(rev_year);

In [6]:
holding_costs = CSV.read("inventory_holding_cost.csv", DataFrame)[:, 2:end]

Row,Year,holding_cost_per_shoe
,Int64,Float64
1,2016,43.92
2,2017,36.75
3,2018,30.22
4,2019,37.57
5,2020,43.73
6,2021,35.65
7,2022,36.63
8,2023,38.44


In [7]:
shipping_costs = CSV.read("transport_costs.csv", DataFrame);

In [8]:
shipping_mapping = shipping_costs[:, 1:2]
shipping_mapping = sort(shipping_mapping, [:producer_region, :consumer_region], rev=[false, false])
shipping_mapping = hcat(DataFrame(Row_Count=1:nrow(shipping_mapping)), shipping_mapping)
shipping_mapping = combine(groupby(shipping_mapping, [:producer_region, :consumer_region])) do sub_df
    DataFrame(Value_mean = first(sub_df.Row_Count))
end
shipping_mapping = unstack(shipping_mapping, :consumer_region, :Value_mean)
shipping_mapping = sort(shipping_mapping, :producer_region)

Row,producer_region,Asia Pacific,Greater China,EMEA,North America,Latin America
,String15,Int64?,Int64?,Int64?,Int64?,Int64?
1,AMERICAS,1,3,2,5,4
2,EMEA,6,8,7,10,9
3,N ASIA,11,13,12,15,14
4,S ASIA,16,18,17,20,19
5,SE ASIA,21,23,22,25,24


In [9]:
production_costs = CSV.read("cost.csv", DataFrame)
replace!(production_costs.Region, "Europe, Middle East, and Africa" => "EMEA");
col_order = [1, 9, 8, 7, 6, 5, 4, 3, 2]
production_costs = production_costs[:, col_order]

Row,Region,yr_2016_shoes,yr_2017_shoes,yr_2018_shoes,yr_2019_shoes,yr_2020_shoes,yr_2021_shoes,yr_2022_shoes,yr_2023_shoes
,String15,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,AMERICAS,32.8035,34.2378,32.9632,36.7434,30.2692,35.9466,30.6661,62.7909
2,EMEA,33.1103,30.902,30.7542,34.9388,31.2383,36.7466,31.2809,60.415
3,N ASIA,32.3776,32.7706,35.5545,36.1969,30.163,38.0421,30.0239,57.1254
4,S ASIA,31.3544,29.1506,34.2267,34.0082,28.5077,36.8746,30.8797,63.0772
5,SE ASIA,31.7476,34.0283,35.8217,34.8603,28.9658,37.3151,31.8701,59.8631


In [10]:
production_costs

Row,Region,yr_2016_shoes,yr_2017_shoes,yr_2018_shoes,yr_2019_shoes,yr_2020_shoes,yr_2021_shoes,yr_2022_shoes,yr_2023_shoes
,String15,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,AMERICAS,32.8035,34.2378,32.9632,36.7434,30.2692,35.9466,30.6661,62.7909
2,EMEA,33.1103,30.902,30.7542,34.9388,31.2383,36.7466,31.2809,60.415
3,N ASIA,32.3776,32.7706,35.5545,36.1969,30.163,38.0421,30.0239,57.1254
4,S ASIA,31.3544,29.1506,34.2267,34.0082,28.5077,36.8746,30.8797,63.0772
5,SE ASIA,31.7476,34.0283,35.8217,34.8603,28.9658,37.3151,31.8701,59.8631


In [66]:
cost_of_shoes_init = 116.5
shoes_price_time = [116.5]
for i in 2:8
    shoes_price_minus_1 = shoes_price_time[i-1]
    shoe_price_current = shoes_price_minus_1 * 1.02
    append!(shoes_price_time, shoe_price_current)
end
shoes_price_time

8-element Vector{Float64}:
 116.5
 118.83
 121.2066
 123.630732
 126.10334664
 128.6254135728
 131.197921844256
 133.82188028114112

In [11]:
# Define sizes
num_suppliers = size(availability)[1] #i
num_consumer_regions = size(demand)[1] # j = c(i) 
num_time = size(availability)[2] - 1;  
num_producer_regions = size(availability)[1] # p(i)

5

### Set-up

In [12]:
# Sets
suppliers = 1:num_suppliers
consumer_regions = 1:num_consumer_regions
producer_regions = 1:num_producer_regions
years = 1:num_time
years_incl_zero = 0:num_time

0:8

In [13]:
# Parameters and input data
A = Matrix(availability[:, 2:end]) #5x8
D = Matrix(demand[:, 2:end]) #5x8
R = Array(rev_year) #8x1
H = Array(holding_costs[:, 2]) #8x1
T = Matrix(shipping_costs[:, 6:end]) #25x8
W = Matrix(shipping_mapping[:, 2:end]) #encodes the prodcuer/consumer region for T 5x5
C = Matrix(production_costs[:, 2:end]); #5x8

### Creating the model

In [14]:
# Define parameters (choose 1 value for now)
alpha = 1
cost_of_shoes = 116.5

116.5

In [67]:
function run_model(alpha, suppliers, consumer_regions, producer_regions, years, years_incl_zero, A, D, R, H, T, W, C)
    # Initialize model
    model = Model(Gurobi.Optimizer);

    # Decision variables
    # X_{i,t} = quantity of shoes produced by supplier i (in producer region p(i)) at time t
    # S_{i,j,t} = quantity of shoes sold to consumer region j at time t, that are produced by supplier i (in producer region p(i))
    # E_{i,t} = holding quantity of shoes by supplier i (in producer region p(i)) at time t
    # M_{i,t} = marginal cost of getting supplier i to produce shoes at time t
    @variable(model, X[suppliers, years_incl_zero] >= 0);
    @variable(model, S[suppliers, consumer_regions, years] >= 0);
    @variable(model, E[suppliers, years_incl_zero] >= 0);
    @variable(model, M[consumer_regions, years] >= 0);


    # Objective function
    @objective(model, Max, 
    sum(sum(sum((shoes_price_time[t]-T[W[i,j],t]) * S[i,j,t] for i in suppliers) for j in consumer_regions) for t in years) 
    - sum(sum((C[i,t]) * X[i,t] + H[t] * E[i,t] + alpha * shoes_price_time[t] * M[i,t] for i in suppliers) for t in years))


    # Constraints
    @constraint(model, initial_production_constraint[i in suppliers], X[i,0] == 0);
    @constraint(model, initial_excess_constraint[i in suppliers], E[i,0] == 0);
    @constraint(model, production_sold_excess_relationship[i in suppliers, t in years], sum(S[i,j,t] for j in consumer_regions) == X[i,t] + E[i,t-1] - E[i,t]);
    @constraint(model, demand_constraint[j in consumer_regions, t in years], sum(S[i,j,t] for i in suppliers) <= D[j,t]);
    @constraint(model, supply_production_constraint[i in suppliers, t in years], X[i,t] <= A[i,t]);
    @constraint(model, unmet_demand[j in consumer_regions, t in years], M[j,t] == D[j,t] - sum(S[i,j,t] for i in suppliers)); # M[i,t] = max{X[i,t] − X[i,t−1], 0}

    #set_optimizer_attribute(model, "NonConvex", 2);
    optimize!(model);

    # Get values
    X_values = Matrix(value.(X))
    S_values = value.(S)
    E_values = Matrix(value.(E))
    M_values = Matrix(value.(M))

    return model, X_values, S_values, E_values, M_values
end

run_model (generic function with 1 method)

In [68]:
model_simple, X_values_simple, S_values_simple, E_values_simple, M_values_simple = run_model(0.5, suppliers, consumer_regions, producer_regions, years, years_incl_zero, A, D, R, H, T, W, C);

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-22
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 170 rows, 330 columns and 810 nonzeros
Model fingerprint: 0xc4ee2005
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+01, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [7e+03, 1e+08]
Presolve removed 50 rows and 10 columns
Presolve time: 0.00s
Presolved: 120 rows, 320 columns, 755 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.2364962e+35   3.720000e+32   1.236496e+05      0s
     135    4.2262938e+10   0.000000e+00   0.000000e+00      0s

Solved in 135 iterations and 0.00 seconds (0.00 work units)
Optimal objective  4.226293781e+10

User-callback calls 183, time in user-callback 0.00 sec


In [69]:
net_profit_simple = objective_value(model_simple)
holding_cost_simple = sum(H[t]*sum(E_values_simple[i,t] for i in suppliers) for t in years)
# sales_profit = net_profit + holding_cost

println("Net profit: ", net_profit_simple)
# println("Sales profit: ", sales_profit)
println("Holding cost: ", holding_cost_simple)

Net profit: 4.226293781077709e10
Holding cost: 4.279356439119997e9


In [18]:
# Plot graphs

# Total costs over t
# Region

# Holding quantity over t

# Herfindalhs over t


In [19]:
# Plot graphs across alphas
# Ave Holding quantity over t
# Ave Herfindalhs over t


## Impose Integrality

In [70]:
function run_model_integer(alpha, suppliers, consumer_regions, producer_regions, years, years_incl_zero, A, D, R, H, T, W, C)
    # Initialize model
    model = Model(Gurobi.Optimizer);

    # Decision variables
    # X_{i,t} = quantity of shoes produced by supplier i (in producer region p(i)) at time t
    # S_{i,j,t} = quantity of shoes sold to consumer region j at time t, that are produced by supplier i (in producer region p(i))
    # E_{i,t} = holding quantity of shoes by supplier i (in producer region p(i)) at time t
    # M_{i,t} = marginal cost of getting supplier i to produce shoes at time t
    @variable(model, X[suppliers, years_incl_zero] >= 0, Int);
    @variable(model, S[suppliers, consumer_regions, years] >= 0, Int);
    @variable(model, E[suppliers, years_incl_zero] >= 0,  Int);
    @variable(model, M[consumer_regions, years] >= 0,  Int);


    # Objective function
    @objective(model, Max, 
    sum(sum(sum((shoes_price_time[t]-T[W[i,j],t]) * S[i,j,t] for i in suppliers) for j in consumer_regions) for t in years) 
    - sum(sum((C[i,t]) * X[i,t] + H[t] * E[i,t] + alpha * shoes_price_time[t] * M[i,t] for i in suppliers) for t in years))


    # Constraints
    @constraint(model, initial_production_constraint[i in suppliers], X[i,0] == 0);
    @constraint(model, initial_excess_constraint[i in suppliers], E[i,0] == 0);
    @constraint(model, production_sold_excess_relationship[i in suppliers, t in years], sum(S[i,j,t] for j in consumer_regions) == X[i,t] + E[i,t-1] - E[i,t]);
    @constraint(model, demand_constraint[j in consumer_regions, t in years], sum(S[i,j,t] for i in suppliers) <= D[j,t]);
    @constraint(model, supply_production_constraint[i in suppliers, t in years], X[i,t] <= A[i,t]);
    @constraint(model, unmet_demand[j in consumer_regions, t in years], M[j,t] == D[j,t] - sum(S[i,j,t] for i in suppliers)); # M[i,t] = max{X[i,t] − X[i,t−1], 0}


    #set_optimizer_attribute(model, "NonConvex", 2);
    optimize!(model);

    

    # Get values
    X_values = Matrix(value.(X))
    S_values = value.(S)
    E_values = Matrix(value.(E))
    M_values = Matrix(value.(M))

    return model, X_values, S_values, E_values, M_values
end

run_model_integer (generic function with 1 method)

In [71]:
model_integer, X_values_integer, S_values_integer, E_values_integer, M_values_integer = run_model_integer(0.5, suppliers, consumer_regions, producer_regions, years, years_incl_zero, A, D, R, H, T, W, C);

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-22
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 170 rows, 330 columns and 810 nonzeros
Model fingerprint: 0x3fba9a1d
Variable types: 0 continuous, 330 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+01, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [7e+03, 1e+08]
Found heuristic solution: objective -1.08645e+11
Presolve removed 90 rows and 60 columns
Presolve time: 0.00s
Presolved: 80 rows, 270 columns, 505 nonzeros
Variable types: 0 continuous, 270 integer (0 binary)
Found heuristic solution: objective -8.43340e+10

Root relaxation: objective 4.226294e+10, 129 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Dep

In [72]:
net_profit_integer = objective_value(model_integer)
holding_cost_integer = sum(H[t]*sum(E_values_integer[i,t] for i in suppliers) for t in years)
# sales_profit = net_profit + holding_cost

println("Net profit: ", net_profit_integer)
# println("Sales profit: ", sales_profit)
println("Holding cost: ", holding_cost_integer)

Net profit: 4.2262937128244514e10
Holding cost: 4.27935538105e9


#### Uncertainty in Availability
Define uncertainty based on random vector for reductions in availability (0-0.25)

In [73]:
random_seed = 123
Random.seed!(random_seed)
reduction_sets = []
for i in 1:8
    reductions = rand(10)/4
    append!(reduction_sets, [reductions])
end
reduction_sets

8-element Vector{Any}:
 [0.22657490969937025, 0.11087343311490114, 0.18641834528484852, 0.12802076000915358, 0.0634622722353774, 0.08353788409547971, 0.1068319702183998, 0.2168868000639895, 0.14547808559691142, 0.07786187517626322]
 [0.18664657921210961, 0.20480021089332517, 0.2385398156196156, 0.21147377058776504, 0.14668717513874724, 0.030453159481336173, 0.03444156044398475, 0.012702072814990473, 0.14489791663460896, 0.20096671475343672]
 [0.17078994140517822, 0.1895508153573528, 0.14295515043555873, 0.13475705210978084, 0.2412520282514698, 0.017677563402534957, 0.0826182988682938, 0.20156204088797453, 0.142728535882112, 0.014275914690461078]
 [0.06047046607640247, 0.11419112779429455, 0.13545158911383187, 0.07519194454809983, 0.1764925093192002, 0.2336974419360583, 0.00705954209984358, 0.061920756771170454, 0.044423767109071866, 0.051053017394202316]
 [0.19273732981823954, 0.1796666268975288, 0.12354912563271633, 0.014195510782407023, 0.05680259989325134, 0.11420297313570327, 0.196

In [74]:
function run_model_integer_uncertainty(alpha, suppliers, consumer_regions, producer_regions, years, years_incl_zero, A, D, R, H, T, W, C)
    # Initialize model
    model = Model(Gurobi.Optimizer);

    # Decision variables
    # X_{i,t} = quantity of shoes produced by supplier i (in producer region p(i)) at time t
    # S_{i,j,t} = quantity of shoes sold to consumer region j at time t, that are produced by supplier i (in producer region p(i))
    # E_{i,t} = holding quantity of shoes by supplier i (in producer region p(i)) at time t
    # M_{i,t} = marginal cost of getting supplier i to produce shoes at time t
    @variable(model, X[suppliers, years_incl_zero] >= 0, Int);
    @variable(model, S[suppliers, consumer_regions, years] >= 0, Int);
    @variable(model, E[suppliers, years_incl_zero] >= 0,  Int);
    #@variable(model, M[consumer_regions, years] >= 0,  Int);
    @variable(model, M[suppliers, years] >= 0,  Int);
    # Define uncertain parameters



    # Objective function
    @objective(model, Max, 
    sum(sum(sum((shoes_price_time[t]-T[W[i,j],t]) * S[i,j,t] for i in suppliers) for j in consumer_regions) for t in years) 
    - sum(sum((C[i,t]) * X[i,t] + H[t] * E[i,t] + alpha * shoes_price_time[t] * M[i,t] for i in suppliers) for t in years))

    # Constraints
    @constraint(model, initial_production_constraint[i in suppliers], X[i,0] == 0);
    @constraint(model, initial_excess_constraint[i in suppliers], E[i,0] == 0);
    @constraint(model, production_sold_excess_relationship[i in suppliers, t in years], sum(S[i,j,t] for j in consumer_regions) == X[i,t] + E[i,t-1] - E[i,t]);
    @constraint(model, time_in_inventory[i in suppliers, t in years], sum(S[i,j,t] for j in consumer_regions) >= E[i,t-1]); #goods can't be in inventory more than 1 year
    @constraint(model, demand_constraint[j in consumer_regions, t in years], sum(S[i,j,t] for i in suppliers) <= D[j,t]);
    @constraint(model, supply_production_constraint[i in suppliers, t in years], X[i,t] <= A[i,t] - A[i,t] * maximum(reduction_sets[t])); #add uncertainty
    @constraint(model, unmet_demand[j in consumer_regions, t in years], M[j,t] == D[j,t] - sum(S[i,j,t] for i in suppliers));
    #@constraint(model, unused_availability[i in suppliers, t in years], M[i,t] >= A[i,t] - A[i,t] * maximum(reduction_sets[t]) - X[i, t]);

    #set_optimizer_attribute(model, "NonConvex", 2);
    optimize!(model);

    # Get values
    X_values = Matrix(value.(X))
    S_values = value.(S)
    E_values = Matrix(value.(E))
    M_values = Matrix(value.(M))

    return model, X_values, S_values, E_values, M_values
end

run_model_integer_uncertainty (generic function with 1 method)

In [91]:
model_uncertainty, X_values_uncertainty, S_values_uncertainty, E_values_uncertainty, M_values_uncertainty = run_model_integer_uncertainty(1, suppliers, consumer_regions, producer_regions, years, years_incl_zero, A, D, R, H, T, W, C);

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-22
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 210 rows, 330 columns and 1050 nonzeros
Model fingerprint: 0x7fabfc9a
Variable types: 0 continuous, 330 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+01, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 1e+08]
Found heuristic solution: objective -2.17291e+11
Presolve removed 125 rows and 125 columns
Presolve time: 0.00s
Presolved: 85 rows, 205 columns, 530 nonzeros
Found heuristic solution: objective -1.88669e+11
Variable types: 0 continuous, 205 integer (0 binary)
Found heuristic solution: objective -1.88669e+11

Root relaxation: objective -5.119854e+10, 92 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objec

In [94]:
net_profit_uncertainty = objective_value(model_uncertainty)
holding_cost_uncertainty = sum(H[t]*sum(E_values_uncertainty[i,t] for i in suppliers) for t in 1:5)
# sales_profit = net_profit + holding_cost

println("Net profit: ", net_profit_uncertainty)
# println("Sales profit: ", sales_profit)
println("Holding cost: ", holding_cost_uncertainty)

Net profit: -5.119854186328842e10
Holding cost: 0.0


In [93]:
M_values_uncertainty

5×8 Matrix{Float64}:
 8.30408e6  1.03417e7  3.3786e6   …  2.30219e7  2.58658e7  2.85839e7
 0.0        0.0        0.0           5.98076e7  6.34154e7  7.08985e7
 2.2309e7   2.50644e7  3.00086e7     4.93391e7  4.64893e7  4.66524e7
 0.0        0.0        0.0           8.30089e6  9.33656e6  8.03565e6
 0.0        0.0        0.0           9.99294e7  1.04923e8  1.27871e8

In [78]:
A .- X_values_uncertainty[:, 2:end]

5×8 Matrix{Float64}:
 9.47547e5  1.05376e6  1.15925e6  …   93146.0   10279.0   25615.0
 1.40688e5  1.56456e5  1.72121e5       1642.0    5096.0    4920.0
 5.58396e6  6.20986e6  6.83157e6     217501.0  261481.0    5776.0
 1.48232e7  1.64847e7  1.81351e7     746534.0   20543.0  412781.0
 1.9504e7   2.16901e7  2.38617e7      29070.0  727692.0       3.78898e6

In [79]:
X_values_uncertainty

5×9 Matrix{Float64}:
 0.0       3.2345e6        3.36377e6  …  43072.0        81212.0
 0.0  480240.0        499434.0           21351.0        15597.0
 0.0       1.90611e7       1.98229e7         1.09577e6  18312.0
 0.0       5.05998e7       5.2622e7      86085.0            1.30877e6
 0.0       6.65777e7       6.92386e7         3.0495e6       1.20135e7

## Compare Models

In [80]:
#Outputs
models = ["Simple", "Integer Solution", "Integer + Uncertainty in Availability"]
net_profit = [net_profit_simple, net_profit_integer, net_profit_uncertainty]
holding_cost = [holding_cost_simple, holding_cost_integer, holding_cost_uncertainty]
output = DataFrame(Model = models, NetProfit = net_profit, HoldingCost = holding_cost)

Row,Model,NetProfit,HoldingCost
,String,Float64,Float64
1,Simple,4.22629e10,4.27936e9
2,Integer Solution,4.22629e10,4.27936e9
3,Integer + Uncertainty in Availability,1.2174e10,0.0


In [82]:
#No uncertainty - only difference is the change in intial inventory
function run_model_after_shock(alpha, suppliers, consumer_regions, producer_regions, years, years_incl_zero, A, D, R, H, T, W, C, X_values_pre, E_values_pre)
    # Initialize model
    model = Model(Gurobi.Optimizer);

    # Decision variables
    # X_{i,t} = quantity of shoes produced by supplier i (in producer region p(i)) at time t
    # S_{i,j,t} = quantity of shoes sold to consumer region j at time t, that are produced by supplier i (in producer region p(i))
    # E_{i,t} = holding quantity of shoes by supplier i (in producer region p(i)) at time t
    # M_{i,t} = marginal cost of getting supplier i to produce shoes at time t
    @variable(model, X[suppliers, years_incl_zero] >= 0, Int);
    @variable(model, S[suppliers, consumer_regions, years] >= 0, Int);
    @variable(model, E[suppliers, years_incl_zero] >= 0,  Int);
    @variable(model, M[consumer_regions, years] >= 0,  Int);


    # Objective function
    @objective(model, Max, 
    sum(sum(sum((shoes_price_time[t]-T[W[i,j],t]) * S[i,j,t] for i in suppliers) for j in consumer_regions) for t in years) 
    - sum(sum((C[i,t]) * X[i,t] + H[t] * E[i,t] + alpha * shoes_price_time[t] * M[i,t] for i in suppliers) for t in years))


    # Constraints
    @constraint(model, initial_production_constraint[i in suppliers], X[i,5] == X_values_pre[i,5]);
    @constraint(model, initial_excess_constraint[i in suppliers], E[i,5] == E_values_pre[i, 5]);
    @constraint(model, production_sold_excess_relationship[i in suppliers, t in years], sum(S[i,j,t] for j in consumer_regions) == X[i,t] + E[i,t-1] - E[i,t]);
    @constraint(model, demand_constraint[j in consumer_regions, t in years], sum(S[i,j,t] for i in suppliers) <= D[j,t]);
    @constraint(model, supply_production_constraint[i in suppliers, t in years], X[i,t] <= A[i,t]);
    @constraint(model, unmet_demand[j in consumer_regions, t in years], M[j,t] == D[j,t] - sum(S[i,j,t] for i in suppliers)); # M[i,t] = max{X[i,t] − X[i,t−1], 0}

    #set_optimizer_attribute(model, "NonConvex", 2);
    optimize!(model);

    

    # Get values
    X_values = Matrix(value.(X))
    S_values = value.(S)
    E_values = Matrix(value.(E))
    M_values = Matrix(value.(M))

    return model, X_values, S_values, E_values, M_values
end

run_model_after_shock (generic function with 1 method)

## Determine Best Parameters
Test various parameters on 2016-2020

In [83]:
#Punishment on unmet demand
alpha_values = [0, 0.1, 0.25, 0.5, 0.75, 0.9, 1];
length(alpha_values)

7

In [84]:
#severity of availability shock
delta_values = [0.1, 0.2, 0.3, 0.4, 0.5];
#(1/0.3)

In [85]:
#Intialize Dictionaries for Results - 2016-2020
X_val = Dict()
S_val = Dict()
E_val = Dict()
M_val = Dict()

Dict{Any, Any}()

In [86]:
#Intialize Dictionaries for Results - 2021-2023
X_val_after = Dict()
S_val_after = Dict()
E_val_after = Dict()
M_val_after = Dict()

Dict{Any, Any}()

In [87]:
#Intialize Dataframe of values
results_16_20 = DataFrame(Model_Type = "", Alpha = 0.0, Net_Profit = 0.0, Holding_Cost = 0.0)
results_21_23 = DataFrame(Model_Type = "", Alpha = 0.0, Uncertainty_Range = 0.0, Net_Profit = 0.0, Holding_Cost = 0.0)

Row,Model_Type,Alpha,Uncertainty_Range,Net_Profit,Holding_Cost
,String,Float64,Float64,Float64,Float64
1,,0.0,0.0,0.0,0.0


In [88]:
for i in 1:length(alpha_values)
    alpha = alpha_values[i]
    #1. Run non-robust model
    model_integer, X_values_integer, S_values_integer, E_values_integer, M_values_integer = run_model_integer(alpha, suppliers, consumer_regions, producer_regions, years, years_incl_zero, A, D, R, H, T, W, C);
    net_profit_integer = objective_value(model_integer)
    holding_cost_integer = sum(H[t]*sum(E_values_integer[z,t] for z in suppliers) for t in 1:5)
    #Add outputs to dictionary
    name = string(alpha) * "_non_robust"
    X_val[name] = X_values_integer
    S_val[name] = S_values_integer
    E_val[name] = E_values_integer
    M_val[name] = M_values_integer
    #Append to DataFrame
    push!(results_16_20, ["Non Robust", alpha, net_profit_integer, holding_cost_integer])
    #2. Run robust model
    model_uncertainty, X_values_uncertainty, S_values_uncertainty, E_values_uncertainty, M_values_uncertainty = run_model_integer_uncertainty(alpha, suppliers, consumer_regions, producer_regions, 1:5, 0:5, A, D, R, H, T, W, C);
    net_profit_uncertainty = objective_value(model_uncertainty)
    holding_cost_uncertainty = sum(H[t]*sum(E_values_uncertainty[z,t] for z in suppliers) for t in 1:5)
    #Add outputs to dictionary
    name = string(alpha) * "_robust"
    X_val[name] = X_values_uncertainty
    S_val[name] = S_values_uncertainty
    E_val[name] = E_values_uncertainty
    M_val[name] = M_values_uncertainty
    #Append to DataFrame
    push!(results_16_20, ["Robust", alpha, net_profit_uncertainty, holding_cost_uncertainty])
    #Evalute for various shocks in 2021-2023
    for j in 1:length(delta_values)
        A_shock = A
        for k in 1:5
            for l in 6:8
                Random.seed!(k+l)
                A_shock[k,l] = round(A_shock[k,l] * (1 - rand()/(1/delta_values[j])))
            end
        end
        #Test Non-Robust Model
        model_after_nonrobust, X_values_after_nonrobust, S_values_after_nonrobust, E_values_after_nonrobust, M_values_after_nonrobust = run_model_after_shock(alpha, suppliers, consumer_regions, producer_regions, 6:8, 5:8, A_shock, D, R, H, T, W, C, X_values_integer, E_values_integer);
        net_profit_after_nonrobust = objective_value(model_after_nonrobust)
        holding_cost_after_nonrobust = sum(H[t]*sum(E_values_after_nonrobust[z,t] for z in suppliers) for t in 1:3)
        push!(results_21_23, ["Non Robust", alpha, delta_values[j], net_profit_after_nonrobust, holding_cost_after_nonrobust])
        #Add outputs to dictionary
        name = string(alpha) * string(delta_values[j]) * "_non_robust"
        X_val_after[name] = X_values_after_nonrobust
        S_val_after[name] = S_values_after_nonrobust
        E_val_after[name] = E_values_after_nonrobust
        M_val_after[name] = M_values_after_nonrobust
        #Test Robust Model
        model_after_robust, X_values_after_robust, S_values_after_robust, E_values_after_robust, M_values_after_robust = run_model_after_shock(alpha, suppliers, consumer_regions, producer_regions,6:8, 5:8, A_shock, D, R, H, T, W, C, X_values_uncertainty, E_values_uncertainty);
        net_profit_after_robust = objective_value(model_after_robust)
        holding_cost_after_robust = sum(H[t]*sum(E_values_after_robust[z,t] for z in suppliers) for t in 1:3)
        push!(results_21_23, ["Robust", alpha, delta_values[j], net_profit_after_robust, holding_cost_after_robust])
        #Add outputs to dictionary
        name = string(alpha) * string(delta_values[j]) * "_robust"
        X_val_after[name] = X_values_after_robust
        S_val_after[name] = S_values_after_robust
        E_val_after[name] = E_values_after_robust
        M_val_after[name] = M_values_after_robust
    end

end
    

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-22
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 170 rows, 330 columns and 810 nonzeros
Model fingerprint: 0xe137250b
Variable types: 0 continuous, 330 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+01, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [7e+03, 1e+08]
Found heuristic solution: objective -0.0000000
Presolve removed 90 rows and 60 columns
Presolve time: 0.00s
Presolved: 80 rows, 270 columns, 505 nonzeros
Variable types: 0 continuous, 270 integer (0 binary)
Found heuristic solution: objective 1.437593e+10

Root relaxation: objective 8.824807e+10, 120 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth

In [89]:
results_16_20

Row,Model_Type,Alpha,Net_Profit,Holding_Cost
,String,Float64,Float64,Float64
1,,0.0,0.0,0.0
2,Non Robust,0.0,8.82481e10,2.95723e8
3,Robust,0.0,6.98145e10,0.0
4,Non Robust,0.1,7.8181e10,2.95723e8
5,Robust,0.1,6.78401e10,0.0
6,Non Robust,0.25,6.35643e10,1.0194e9
7,Robust,0.25,6.48784e10,0.0
8,Non Robust,0.5,3.9711e10,2.15566e9
9,Robust,0.5,5.99423e10,0.0


In [95]:
results_21_23[25:51, :]

Row,Model_Type,Alpha,Uncertainty_Range,Net_Profit,Holding_Cost
,String,Float64,Float64,Float64,Float64
1,Robust,0.25,0.2,-2.39512e10,0.0
2,Non Robust,0.25,0.3,-2.26184e10,3.90944e8
3,Robust,0.25,0.3,-2.4043e10,0.0
4,Non Robust,0.25,0.4,-2.27322e10,3.90944e8
5,Robust,0.25,0.4,-2.41567e10,0.0
6,Non Robust,0.25,0.5,-2.28614e10,3.90944e8
7,Robust,0.25,0.5,-2.42859e10,0.0
8,Non Robust,0.5,0.1,-4.57538e10,8.38438e8
9,Robust,0.5,0.1,-4.94218e10,0.0


Model 2016-2020 with original data

In [96]:
model_preshock, X_values_preshock, S_values_preshock, E_values_preshock, M_values_preshock = run_model_integer_uncertainty(0.5, 
    suppliers, consumer_regions, producer_regions, 1:5, 0:5, A[:, 1:5], D[:, 1:5], R[1:5], H[1:5], T[:, 1:5], W, C[:, 1:5]);

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-22
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 135 rows, 210 columns and 660 nonzeros
Model fingerprint: 0x979f5b1a
Variable types: 0 continuous, 210 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+01, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+05, 9e+07]
Found heuristic solution: objective -5.77360e+10
Presolve removed 65 rows and 45 columns
Presolve time: 0.00s
Presolved: 70 rows, 165 columns, 430 nonzeros
Variable types: 0 continuous, 165 integer (0 binary)
Found heuristic solution: objective -3.77894e+10

Root relaxation: objective 5.994235e+10, 61 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Dept

In [97]:
net_profit_preshock = objective_value(model_preshock)
holding_cost_preshock = sum(H[t]*sum(E_values_preshock[i,t] for i in suppliers) for t in 1:5)
# sales_profit = net_profit + holding_cost

println("Net profit: ", net_profit_preshock)
# println("Sales profit: ", sales_profit)
println("Holding cost: ", holding_cost_preshock)

Net profit: 5.994234651244529e10
Holding cost: 0.0


Model 2021-2023 with Supply Chain Shock

In [98]:
function run_model_shock(alpha, suppliers, consumer_regions, producer_regions, years, years_incl_zero, A, D, R, H, T, W, C)
    # Initialize model
    model = Model(Gurobi.Optimizer);

    # Decision variables
    # X_{i,t} = quantity of shoes produced by supplier i (in producer region p(i)) at time t
    # S_{i,j,t} = quantity of shoes sold to consumer region j at time t, that are produced by supplier i (in producer region p(i))
    # E_{i,t} = holding quantity of shoes by supplier i (in producer region p(i)) at time t
    # M_{i,t} = marginal cost of getting supplier i to produce shoes at time t
    @variable(model, X[suppliers, years_incl_zero] >= 0, Int);
    @variable(model, S[suppliers, consumer_regions, years] >= 0, Int);
    @variable(model, E[suppliers, years_incl_zero] >= 0,  Int);
    @variable(model, M[consumer_regions, years] >= 0, Int);
    # Define uncertain parameters



    # Objective function
    @objective(model, Max, 
    sum(sum(sum((cost_of_shoes-T[W[i,j],t]) * S[i,j,t] for i in suppliers) for j in consumer_regions) for t in years) 
    - sum(sum((C[i,t]) * X[i,t] + H[t] * E[i,t] + alpha * cost_of_shoes * M[i,t] for i in suppliers) for t in years))


    # Constraints
    @constraint(model, initial_production_constraint[i in suppliers], X[i,5] == X_values_preshock[i, 5]);
    @constraint(model, initial_excess_constraint[i in suppliers], E[i,5] == E_values_preshock[i, 5]);
    @constraint(model, production_sold_excess_relationship[i in suppliers, t in years], sum(S[i,j,t] for j in consumer_regions) == X[i,t] + E[i,t-1] - E[i,t]);
    @constraint(model, time_in_inventory[i in suppliers, t in years], sum(S[i,j,t] for j in consumer_regions) >= E[i,t-1]); #goods can't be in inventory more than 1 year
    @constraint(model, demand_constraint[j in consumer_regions, t in years], sum(S[i,j,t] for i in suppliers) <= 2*(D[j,t]/avg_price * sum(price_list_abs_dif)) - D[j,t]);
    @constraint(model, supply_production_constraint[i in suppliers, t in years], X[i,t] <= A[i,t]);
    @constraint(model, unmet_demand[j in consumer_regions, t in years], M[j,t] == D[j,t] - sum(S[i,j,t] for i in suppliers)); # M[i,t] = max{X[i,t] − X[i,t−1], 0}

    
    #set_optimizer_attribute(model, "NonConvex", 2);
    optimize!(model);

    # Get values
    X_values = Matrix(value.(X))
    S_values = value.(S)
    E_values = Matrix(value.(E))
    M_values = Matrix(value.(M))

    return model, X_values, S_values, E_values, M_values
end

run_model_shock (generic function with 1 method)

In [99]:
model_shock, X_values_shock, S_values_shock, E_values_shock, M_values_shock = run_model_shock(0.5, 
    suppliers, consumer_regions, producer_regions, 6:8, 5:8, A_shock, D, R, H, T, W, C);

LoadError: UndefVarError: `A_shock` not defined

In [46]:
net_profit_shock = objective_value(model_shock)
holding_cost_shock = sum(H[t]*sum(E_values_shock[i,t] for i in suppliers) for t in 1:3)
# sales_profit = net_profit + holding_cost

println("Net profit: ", net_profit_shock)
# println("Sales profit: ", sales_profit)
println("Holding cost: ", holding_cost_shock)

LoadError: UndefVarError: `model_shock` not defined

In [47]:
net_profit_pre_post = net_profit_shock + net_profit_preshock
holding_cost_pre_post = holding_cost_shock + holding_cost_preshock
# sales_profit = net_profit + holding_cost

println("Net profit: ", net_profit_pre_post)
# println("Sales profit: ", sales_profit)
println("Holding cost: ", holding_cost_pre_post)

LoadError: UndefVarError: `net_profit_shock` not defined

In [48]:
#Outputs
models = ["No Shock", "Mild Shock (~25% Reduction)"]
net_profit = [net_profit_uncertainty, net_profit_pre_post ]
holding_cost = [holding_cost_uncertainty, holding_cost_pre_post]
output = DataFrame(Model = models, NetProfit = net_profit, HoldingCost = holding_cost)

LoadError: UndefVarError: `net_profit_pre_post` not defined